In [ ]:
import random
import nltk
import json
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

import logging

from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [ ]:
!pip install python-telegram-bot --upgrade

In [ ]:
    with open('/content/BOT_CONFIG.json', 'r') as f:
  BOT_CONFIG = json.load(f)

## День 2

In [ ]:
X = []
Y = []
for intent in BOT_CONFIG['intents'].keys(): # сборка текстов и интентов
    try: # отсев не валидных данных из файла
      if intent != 'other':
        for example in BOT_CONFIG['intents'][intent]['examples']:
            X.append(example)
            Y.append(intent)
    except: # вывод данных с ошибками
      ;

In [ ]:
len(X), len(Y) # сверка, что для каждого текста есть интент

(1173, 1173)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( # разбивка датасета на тестовый и для обучения
    X, Y, test_size=0.2, random_state=42)

In [ ]:
vectorizer = CountVectorizer(ngram_range=(2, 6), analyzer='char_wb') # определение параметров для векторизации
X_train_vectorized = vectorizer.fit_transform(X_train) # веторизация датасета для обучения
X_test_vectorized = vectorizer.transform(X_test) # векторизация датасета для теста
len(vectorizer.get_feature_names()) # размер словаря для обучения

11842

In [ ]:
clf = LogisticRegression().fit(X_train_vectorized, y_train) # обучение модели на основе логистической регрессии

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
clf.score(X_train_vectorized, y_train)

0.8486140724946695

In [ ]:
clf.score(X_test_vectorized, y_test)

0.26382978723404255

In [ ]:
def get_intents_by_model(text):
  return clf.predict(vectorizer.transform([text]))[0]

In [ ]:
def bot_response(text): # возвращает ответ бота на взодящий запрос
  intent = get_intents_by_model(text) # получение от модели интента
  return random.choice(BOT_CONFIG['intents'][intent]['responces']) # возврат случайного ответа из вариантов для указанного интента

In [ ]:
# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    update.message.reply_text(bot_response(update.message.text))


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("1904862824:AAE1dgD3cddICt7oVNDwFVtQ9IG_0x0k9aE")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


In [ ]:
main()

2021-08-12 13:44:37,524 - apscheduler.scheduler - INFO - Scheduler started
2021-08-12 13:49:33,986 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/telegram/ext/dispatcher.py", line 555, in process_update
    handler.handle_update(update, self, check, context)
  File "/usr/local/lib/python3.7/dist-packages/telegram/ext/handler.py", line 198, in handle_update
    return self.callback(update, context)
  File "<ipython-input-31-77c2e036b80e>", line 27, in echo
    update.message.reply_text(bot_response(update.message.text))
  File "<ipython-input-26-508329a80179>", line 3, in bot_response
    return random.choice(BOT_CONFIG['intents'][intent]['responces']) # возврат случайного ответа из вариантов для указанного интента
KeyError: 'responces'
2021-08-12 13:49:51,207 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most r